In [1]:
import os
from dotenv import load_dotenv, find_dotenv
import warnings

_ = load_dotenv(find_dotenv())
warnings.filterwarnings('ignore')

In [7]:
from IPython.display import display, Markdown

## Importing an LLM

In [2]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0,
    max_retries=2,
    # other params...
)

## Chatting with an LLM
The good thing about langchain is that llms are only `invoke`-ed. There is no need to remember any other method to interact with an LLM

Invoke on a single string

In [32]:
response = llm.invoke("Hi!")

In [33]:
display(Markdown(response.content))

Hello! How can I assist you today? Let's chat about anything you'd like. 😊

Invoke on a list of messages

In [4]:
messages = [
    (
        "system",
        "You are a helpful assistant."
    ),
    ("human", "Why is the sky blue?")
]

In [5]:
response = llm.invoke(messages)

In [6]:
response

AIMessage(content="The sky appears blue due to a process called Rayleigh scattering. As light from the sun reaches Earth's atmosphere, it is made up of different colors, which are essentially different wavelengths of light. Shorter wavelengths (like blue and violet) are scattered more by the tiny molecules and particles in the atmosphere because they travel in shorter, smaller waves. This is why we perceive the sky as blue most of the time.\n\nHowever, you might wonder why the sky doesn't appear violet, since violet light has an even shorter wavelength than blue light. The sky appears blue, not violet, because the sun emits more blue light than violet light, and also because our eyes are more sensitive to blue light. Additionally, some of the violet light gets absorbed by the atmosphere, further tipping the balance toward blue.\n\nDuring sunrise or sunset, the light has to pass through more of Earth's atmosphere, which scatters more of the blue and green light away, and we're left with

In [8]:
str(response)

'content="The sky appears blue due to a process called Rayleigh scattering. As light from the sun reaches Earth\'s atmosphere, it is made up of different colors, which are essentially different wavelengths of light. Shorter wavelengths (like blue and violet) are scattered more by the tiny molecules and particles in the atmosphere because they travel in shorter, smaller waves. This is why we perceive the sky as blue most of the time.\\n\\nHowever, you might wonder why the sky doesn\'t appear violet, since violet light has an even shorter wavelength than blue light. The sky appears blue, not violet, because the sun emits more blue light than violet light, and also because our eyes are more sensitive to blue light. Additionally, some of the violet light gets absorbed by the atmosphere, further tipping the balance toward blue.\\n\\nDuring sunrise or sunset, the light has to pass through more of Earth\'s atmosphere, which scatters more of the blue and green light away, and we\'re left with 

Notice that using the `str` method does not return the content. It just returns the entire output in string format. So we have to access the content attribute directly.

In [9]:
display(Markdown(response.content))

The sky appears blue due to a process called Rayleigh scattering. As light from the sun reaches Earth's atmosphere, it is made up of different colors, which are essentially different wavelengths of light. Shorter wavelengths (like blue and violet) are scattered more by the tiny molecules and particles in the atmosphere because they travel in shorter, smaller waves. This is why we perceive the sky as blue most of the time.

However, you might wonder why the sky doesn't appear violet, since violet light has an even shorter wavelength than blue light. The sky appears blue, not violet, because the sun emits more blue light than violet light, and also because our eyes are more sensitive to blue light. Additionally, some of the violet light gets absorbed by the atmosphere, further tipping the balance toward blue.

During sunrise or sunset, the light has to pass through more of Earth's atmosphere, which scatters more of the blue and green light away, and we're left with the warmer colors of sunrise and sunset, like red, orange, and yellow.

### Using the chain abstraction - this is where the magic happens

In [10]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    # Now we instantiate our list of messages
    [
        (
            "system",
            "You are a helpful assistant."
        ),
        ("human", "{input}") #input is a placeholder
    ]
)

Notice that we defined a placeholder `{input}`. When invoking an llm, you must ensure that there is an argument named `input` or there will be an error. The invocation of the chain is typically done using a dictionary

#### Defining our chain

Langchain allows for users to use the pipe `|` operator to quickly define a chain. This allows for pretty much anything to be chainable!

In [11]:
chain = prompt | llm 

In [12]:
response = chain.invoke({
    'input': 'Why is the sky blue?'
})

In [13]:
response

AIMessage(content="The sky appears blue due to a process called Rayleigh scattering. As light from the sun reaches Earth's atmosphere, it is made up of different colors, which are essentially different wavelengths of light. Shorter wavelengths (like blue and violet) are scattered more by the tiny molecules and particles in the atmosphere because they travel in shorter, smaller waves. This is why we perceive the sky as blue most of the time.\n\nHowever, you might wonder why the sky doesn't appear violet, since violet light has an even shorter wavelength than blue light. The sky appears blue, not violet, because the sun emits more blue light than violet light, and also because our eyes are more sensitive to blue light. Additionally, some of the violet light gets absorbed by the atmosphere, further tipping the balance toward blue.\n\nDuring sunrise or sunset, the light has to pass through more of Earth's atmosphere, which scatters more of the blue and green light away, and we're left with

It can be quite bothersome to continue typing `response.content` to access the string. Thankfully Langchain has an output parser convenience function that we can call

In [14]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = prompt | llm | parser

response = chain.invoke(
    {"input": "Why is the sky blue?"}
)

In [15]:
display(Markdown(response))

The sky appears blue due to a process called Rayleigh scattering. As light from the sun reaches Earth's atmosphere, it is made up of different colors, which are essentially different wavelengths of light. Shorter wavelengths (like blue and violet) are scattered more by the tiny molecules and particles in the atmosphere because they travel in shorter, smaller waves. This is why we perceive the sky as blue most of the time.

However, you might wonder why the sky doesn't appear violet, since violet light has an even shorter wavelength than blue light. The sky appears blue, not violet, because the sun emits more blue light than violet light, and also because our eyes are more sensitive to blue light. Additionally, some of the violet light gets absorbed by the atmosphere, further tipping the balance toward blue.

During sunrise or sunset, the light has to pass through more of Earth's atmosphere, which scatters more of the blue and green light away, and we're left with the warmer colors of sunrise and sunset, like red, orange, and yellow.

#### Adding message history
You can continue to append messages to the list of messages, or use Langchain's high level abstraction for this!

In [16]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

We will have to define our history store and a simple function to return the correct chat history given an id

In [18]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [21]:
from langchain_core.prompts import MessagesPlaceholder

# Define our base runnable again
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}."
        ),
        MessagesPlaceholder(variable_name="messages") #placeholder
    ]
)
chain = prompt | llm | parser

# Wrraping the runnable on top of another runnable
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages" #Must be the same as the placeholder
)

In [22]:
# Set a session_id within the config
config = {"configurable": {"session_id": "abc123"}} 

In [23]:
from langchain_core.messages import HumanMessage

response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content = "Hi my name is Titus! Why is the sky blue?")],
        "language": "Chinese"
    },
    config = config
)

In [25]:
display(Markdown(response))

你好，Titus！天空之所以是蓝色的，主要是由于大气层中的光散射现象。当太阳光进入地球大气层时，光线会与空气分子和微小颗粒发生碰撞，导致光线发生散射。蓝色光的波长较短，散射的程度比其他颜色的光更强烈，因此我们看到的天空是蓝色的。这种现象被称为瑞利散射。

In [26]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content = "What's my name again?")],
        "language": "French"
    },
    config = config
)

In [27]:
display(Markdown(response))

Bien sûr, je suis là pour t'aider. Ton nom est Titus.

Let's take a look at our chat history!

In [28]:
print(store)

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi my name is Titus! Why is the sky blue?', additional_kwargs={}, response_metadata={}), AIMessage(content='你好，Titus！天空之所以是蓝色的，主要是由于大气层中的光散射现象。当太阳光进入地球大气层时，光线会与空气分子和微小颗粒发生碰撞，导致光线发生散射。蓝色光的波长较短，散射的程度比其他颜色的光更强烈，因此我们看到的天空是蓝色的。这种现象被称为瑞利散射。', additional_kwargs={}, response_metadata={}), HumanMessage(content="What's my name again?", additional_kwargs={}, response_metadata={}), AIMessage(content="Bien sûr, je suis là pour t'aider. Ton nom est Titus.", additional_kwargs={}, response_metadata={})])}


## Structured Prediction

In [29]:
from pydantic import BaseModel
from typing import List

class Exercise(BaseModel):
    """Generates a workout plan given a focus area"""
    exercise: str
    number_of_reps: int
    number_of_sets: int

class WorkoutPlan(BaseModel):
    exercises: List[Exercise]

In [30]:
structured_llm = llm.with_structured_output(WorkoutPlan)

In [35]:
response = structured_llm.invoke("Suggest a workout plan for biceps")

In [36]:
response

WorkoutPlan(exercises=[Exercise(exercise='Barbell Curl', number_of_reps=10, number_of_sets=3), Exercise(exercise='Hammer Curl', number_of_reps=10, number_of_sets=3), Exercise(exercise='Concentration Curl', number_of_reps=10, number_of_sets=3)])

## Zero shot classification!
Sometimes known as 'routing'

In [37]:
from typing import Literal

class DocumentFolder(BaseModel):
    """Returns only one output folder name given a document name"""
    
    folder_name: Literal[
        "finance",
        "presentations",
        "sketchbook",
        "code"
    ]

In [40]:
folder_sorter = llm.with_structured_output(DocumentFolder)
response = folder_sorter.invoke("Where should I store the file 'llama_sketch.png'?")

In [41]:
response

DocumentFolder(folder_name='sketchbook')

### With metdata, we can do this via semantic similarity!

In [43]:
presentation = "choose this option if the input document from the user's query has a .pptx or .ppt extension. Here is the user's query: {query}"
python_code = "choose this option if the input document from the user's query has a .py or .ipynb extension. Here is the user's query: {query}"

But first we have to define our own router function

In [62]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.utils.math import cosine_similarity
from langchain_core.prompts import PromptTemplate

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

prompt_templates = [presentation, python_code]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    if most_similar == presentation:
        return "presentation"
    return "python_code"

In [63]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

route_chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
)

In [64]:
result = route_chain.invoke("Where should I keep this file: abc.py?")

In [65]:
result

'python_code'

By now you should be starting to get the sense of how flexible Langchain is! Chains can be combined with other chains to form more chains. And a chain can be anything!

## Calling tools
Langchain allows for any function to be used as a tool for an LLM. Do check that the LLM is a function-calling LLM first!

To register a function as a tool, simply add the `@tool` decorator on the function of interest!

In [67]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies two integers and returns the resulting integer"""
    return a*b

@tool
def mystery(a: int, b: int) -> int:
    """Mystery function on two integers"""
    return a * b + a + b

@tool
def add( a: int, b: int) -> int:
    """Use this tool to add two integers together"""
    return a + b

@tool
def minus(a: int, b: int) -> int:
    """Use this tool to return the difference between two integers"""
    return a - b

@tool
def divide(a: int, b: int) -> int:
    """Use this tool to perform integer division between two integers"""
    return a //b

In [68]:
tools = [add, multiply, mystery, minus, divide]

In [69]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools = llm.bind_tools(tools)
response = llm_with_tools.invoke(query)

In [70]:
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '0AXkLCfCQ', 'type': 'function', 'function': {'name': 'multiply', 'arguments': '{"a": 3, "b": 12}'}}, {'id': 'NpiIIdDDt', 'type': 'function', 'function': {'name': 'add', 'arguments': '{"a": 11, "b": 49}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 390, 'total_tokens': 439, 'completion_tokens': 49}, 'model': 'mistral-large-latest', 'finish_reason': 'tool_calls'}, id='run-83123bb6-41a3-4103-a456-2cdb3d63389f-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '0AXkLCfCQ', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'NpiIIdDDt', 'type': 'tool_call'}], usage_metadata={'input_tokens': 390, 'output_tokens': 49, 'total_tokens': 439})

Hey there's no response here! What's going on? 
> Langchain's abstraction means the llm only 'chose' the tool it was going to use without actually executing it. You'll also notice that the tools have `ids`!

These are the tools that were called

In [71]:
response.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': '0AXkLCfCQ',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'NpiIIdDDt',
  'type': 'tool_call'}]

Now let's execute the tool outputs and pass the outputs back to the llm. 

In [73]:
messages = [HumanMessage(query)]
response = llm_with_tools.invoke(messages)
messages.append(response)

In [74]:
messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'HCsQ8cvM0', 'type': 'function', 'function': {'name': 'multiply', 'arguments': '{"a": 3, "b": 12}'}}, {'id': 'UiZ7CKbna', 'type': 'function', 'function': {'name': 'add', 'arguments': '{"a": 11, "b": 49}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 390, 'total_tokens': 439, 'completion_tokens': 49}, 'model': 'mistral-large-latest', 'finish_reason': 'tool_calls'}, id='run-4ab20826-bc73-4e11-a7a8-c3861dc3d45d-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'HCsQ8cvM0', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'UiZ7CKbna', 'type': 'tool_call'}], usage_metadata={'input_tokens': 390, 'output_tokens': 49, 'total_tokens': 439})]

In [76]:
from langchain_core.messages import ToolMessage

for tool_call in response.tool_calls:
    selected_tool = {
        "add": add,
        "mystery": mystery,
        "multiply": multiply,
        "divide": divide,
        "minus": minus,
    }[tool_call['name'].lower()]
    print(selected_tool)
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

name='multiply' description='Multiplies two integers and returns the resulting integer' args_schema=<class 'langchain_core.utils.pydantic.multiply'> func=<function multiply at 0x342e8a700>
name='add' description='Use this tool to add two integers together' args_schema=<class 'langchain_core.utils.pydantic.add'> func=<function add at 0x342e8a7a0>


In [77]:
messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'HCsQ8cvM0', 'type': 'function', 'function': {'name': 'multiply', 'arguments': '{"a": 3, "b": 12}'}}, {'id': 'UiZ7CKbna', 'type': 'function', 'function': {'name': 'add', 'arguments': '{"a": 11, "b": 49}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 390, 'total_tokens': 439, 'completion_tokens': 49}, 'model': 'mistral-large-latest', 'finish_reason': 'tool_calls'}, id='run-4ab20826-bc73-4e11-a7a8-c3861dc3d45d-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'HCsQ8cvM0', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'UiZ7CKbna', 'type': 'tool_call'}], usage_metadata={'input_tokens': 390, 'output_tokens': 49, 'total_tokens': 439}),
 ToolMessage(content='36', tool_call_id='HCsQ8cvM0'),
 ToolMessage(content='60', tool_call_id='UiZ7CKbna')]

Our list of messages has now been updated with the tool calls and the results of each tool call! Now we just need to send this whole thing to the llm

In [78]:
final_reply = llm_with_tools.invoke(messages)

In [79]:
final_reply

AIMessage(content='The result of 3 * 12 is 36.\n\nThe result of 11 + 49 is 60.', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 495, 'total_tokens': 526, 'completion_tokens': 31}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-55f80c86-9ee3-4bbf-9092-d1607e005af5-0', usage_metadata={'input_tokens': 495, 'output_tokens': 31, 'total_tokens': 526})

Ah! now there's actual `content` in the result

In [80]:
display(Markdown(final_reply.content))

The result of 3 * 12 is 36.

The result of 11 + 49 is 60.

Langchain does require the user to define quite a lot of things but there is an advantage to this approach. As an example, we can do few-shot prompting by adding examples into the messages list! Few shot-prompting can be quite useful for an llm in learning how to use very complex tools

> The examples are copied from https://python.langchain.com/v0.1/docs/modules/model_io/chat/function_calling/#response-reading-tool-calls-from-model-output

In [81]:
from langchain_core.messages import AIMessage

examples = [
    HumanMessage(
        "What's the product of 317253 and 128472 plus four", name="example_user"
    ),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {"name": "multiply", "args": {"x": 317253, "y": 128472}, "id": "1"}
        ],
    ),
    ToolMessage("16505054784", tool_call_id="1"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[{"name": "add", "args": {"x": 16505054784, "y": 4}, "id": "2"}],
    ),
    ToolMessage("16505054788", tool_call_id="2"),
    AIMessage(
        "The product of 317253 and 128472 plus four is 16505054788",
        name="example_assistant",
    ),
]

In [82]:
system = """You are bad at math but are an expert at using a calculator."""

few_shot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        *examples,
        ("human", "{query}"),
    ]
)

In [83]:
chain = {"query": RunnablePassthrough()} | few_shot_prompt | llm_with_tools

In [91]:
# As before:

messages=[HumanMessage("What is 119 minus 8 times 20?")]
response = llm_with_tools.invoke(messages)
messages.append(response)
for tool_call in response.tool_calls:
    selected_tool = {
        "add": add,
        "mystery": mystery,
        "multiply": multiply,
        "divide": divide,
        "minus": minus,
    }[tool_call['name'].lower()]
    print(selected_tool)
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
    print(f"Appended tool output: {messages[-1]}")

name='multiply' description='Multiplies two integers and returns the resulting integer' args_schema=<class 'langchain_core.utils.pydantic.multiply'> func=<function multiply at 0x342e8a700>
Appended tool output: content='160' tool_call_id='mbk6IkI2A'


## RAG
Unlike LlamaIndex, langchain does not offer their own vector store abstraction. You must provide a vector store for the llm to access. Let's use Chroma to do this!

In [100]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [102]:
loader = TextLoader("../../LlamaIndex/data/paul_graham/paul_graham_essay.txt")

In [103]:
docs = loader.load()

In [105]:
len(docs)

1

Yup! The entire text file is read as one document. You still must split it up!

In [106]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=20)
splits = text_splitter.split_documents(docs)

In [107]:
len(splits)

101

Instantiate the vector store and ingest the splits
> Note: This instantiates an in-memory Chroma vector store instance

In [122]:
from langchain_chroma import Chroma

vector_store = Chroma.from_documents(
    documents = splits,
    embedding = embeddings,
)

Once ingested. The first thing we need to do is set the vector store as a retriever. Langchain allows you to do this through the `.as_retriever()` method. We'll also pull a frequently used system prompt for RAG systems.

In [123]:
from langchain import hub

retriever = vector_store.as_retriever()

# Pull prompt (or you can define your own system prompt)
prompt = hub.pull("rlm/rag-prompt")

In [124]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

Define a utility function to format documents extracted from retrieval

In [125]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

#### Define our RAG chain
Now this part is different from the easier "chains" we made. We need to specifically tell the chain that the "context" for the llm to look at is a small chain (defined to be `retriever | format_docs`). We then also need to have an input key that is a placeholder within the runnable (hence the `RunnablePassthrough`)

In [126]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [127]:
response = rag_chain.invoke("Did Paul Graham write code?")

In [128]:
display(Markdown(response))

Yes, Paul Graham wrote code. He mentioned working extensively on a project called Bel and developing a new dialect of Lisp called Arc. He also wrote all of Y Combinator's internal software in Arc.

## Instantiate an agent
Once again there are many different agent types. Let's just call the simplest!

In [140]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_tool_calling_agent(llm, tools, prompt)

In [141]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [142]:
agent_executor.invoke({"input": "What is the mystery function's output on 57 and 26?"})



> Entering new AgentExecutor chain...

Invoking: `mystery` with `{'a': 57, 'b': 26}`


1565The mystery function's output on 57 and 26 is 1565.

> Finished chain.


{'input': "What is the mystery function's output on 57 and 26?",
 'output': "The mystery function's output on 57 and 26 is 1565."}